# genewiz_rnaseq - rna_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [1]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/processed_raw_reads
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/jsons
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs

Save metadata file

In [2]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt
Sequencing core user	Sequencing core project	Sequencing core password	Sequencing core library name	Name	Paired-end or single-end	Genome	Library type	Strand specificity	With ercc spike-in
XXXXXXX	XXXXXXX	XXXXXXX	381	mm10Liver.rnaseq.non-targeting.rep1	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	382	mm10Liver.rnaseq.non-targeting.rep2	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	383	mm10Liver.rnaseq.non-targeting.rep3	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	384	mm10Liver.rnaseq.PBS.rep1	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	385	mm10Liver.rnaseq.non-targeting.rep4	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	386	mm10Liver.rnaseq.PDX1.rep1	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	387	mm10Liver.rnaseq.PDX1.rep2	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	388	mm10Liver.rnaseq.PDX1.rep3	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	389	mm10Liver.rnaseq.PDX1.rep4	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	3810	mm10Liver.rnaseq.PBS.rep2	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	3811	mm10Liver.rnaseq.PBS.rep3	PE	mm10	RNA-seq	unstranded	False
XXXXXXX	XXXXXXX	XXXXXXX	3812	mm10Liver.rnaseq.PBS.rep4	PE	mm10	RNA-seq	unstranded	False


Overwriting /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt


#### Download FASTQ from sftp
Create file to download FASTQ files

In [1]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/download_genewiz_rnaseq.sh
#!/bin/bash
METADATA=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt
DATA_HOME=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq
mkdir -p ${DATA_HOME}/raw_reads/

read CORE_USER CORE_PROJECT CORE_PASS <<< $(cut -f 1,2,3 $METADATA | tail -n+2 | sort -u)
cd ${DATA_HOME}/raw_reads/
CORE_USER="lastpass"
CORE_PASS="lastpass"
GENEWIZ_PROJECT="30-295498228"
lftp -u ${CORE_USER},${CORE_PASS} -e "mirror --parallel=16 --only-missing ${GENEWIZ_PROJECT}/00_fastq; exit" "sftp://${CORE_USER}@sftp.genewiz.com"
exit


Writing /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/download_genewiz_rnaseq.sh


Execute file to download files

In [ ]:
%%script --out blocking_job_str bash
sbatch -o /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/genewiz_rnaseq_download_fastq_files.out \
 -p new,all \
 --wrap="ssh aeb84@Hardac-xfer.genome.duke.edu 'sh /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/download_genewiz_rnaseq.sh' "

Extract blocking job id

In [ ]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Ungzip FASTQ files

In [12]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/ungzip_genewiz_rnaseq.sh
#!/bin/bash
ORDER=genewiz_rnaseq
PROJECT_HOME=/data/reddylab/Alex/collab/20190701_Matt
METADATA=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt
RAW_DATA_DIR=${PROJECT_HOME}/data/rna_seq/raw_reads/${ORDER}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}. Fix it"
    exit 1
fi

if [[ ! -e ${RAW_DATA_DIR} ]];
then
    RAW_DATA_DIR=${PROJECT_HOME}/data/rna_seq/raw_reads/${ORDER^^}
fi
PROCESSED_DATA_DIR=${PROJECT_HOME}/data/rna_seq/processed_raw_reads/${ORDER}

mkdir -p ${PROCESSED_DATA_DIR}

sample=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} |\
            awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');

for ii in $(/bin/find ${RAW_DATA_DIR}/ -type f | /bin/grep "${sample/ /}_.*.gz");
do
    sample_gz_file=$(basename ${ii})
#     if [[ ! -e ${PROCESSED_DATA_DIR}/${sample_gz_file/.gz/} ]];
#     then
        gzip -cd ${ii} > ${PROCESSED_DATA_DIR}/${sample_gz_file/.gz/};
#     fi
done

Overwriting /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/ungzip_genewiz_rnaseq.sh


Execute file to ungzip FASTQ files

In [3]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/genewiz_rnaseq_ungzip_fastq_files_%a.out \
 -p new,all \
 --array 10,11 \
 /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/ungzip_genewiz_rnaseq.sh

Extract blocking job id

In [4]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Merge lanes of FASTQ files

In [11]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/merge_lanes_genewiz_rnaseq.sh
#!/bin/bash
#SBATCH --array=0-12%20
ORDER=genewiz_rnaseq
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt

mkdir -p ${PROCESSED_DATA_DIR}
cd ${PROCESSED_DATA_DIR}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}"
    exit 1
fi

name_header=$(/bin/grep -Poi "\tname\t" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Library Name column not found in ${METADATA}"
    exit 1
fi
name_header=$(echo ${name_header} | cut -f2)

seq_type_header=$(head -1 ${METADATA} | /bin/grep -Poi "paired.?end.?or.?single.?end")
if [[ $? == 1 ]];
then
    echo -e "ERROR: Paired-end or single-end column not found in ${METADATA}"
    exit 1
fi

sample_seq_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
sample_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
seq_type=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_type_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');


for read_pair in R1 R2;
do
    sample_files=$(/bin/ls ${sample_seq_name/ /}_${read_pair}_*)
    if [[ ${read_pair} == "R1" || (${seq_type/ /} == "PE" || ${seq_type/ /} == "pe") ]];
    then
        # Merge all lanes
        merged=$(echo ${sample_files} | awk '{print $1}')
#         cat ${sample_files} > ${merged};
#         rm -f ${sample_files}

        # Rename samples with our sample Names
        dest_filename=$(basename ${merged} | sed -r 's/\_(R[123])\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
        mv ${merged} ${dest_filename}

        cleaned_dest_filename=${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

        if [[ ${seq_type/ /} == "SE" || ${seq_type/ /} == "se" ]];
        then
            cleaned_dest_filename=${cleaned_dest_filename/.R1/}
        fi
        
        mv ${dest_filename} ${cleaned_dest_filename}
    fi
done


Overwriting /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/merge_lanes_genewiz_rnaseq.sh


Execute file to merge lanes of FASTQ files

In [6]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/genewiz_rnaseq_merge_fastq_files_%a.out \
 -p new,all \
 --depend afterok:$1 \
 --array 10-11 \
 /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/merge_lanes_genewiz_rnaseq.sh

Extract blocking job id

In [7]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create JSON files for CWL pipeline files

In [4]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/cwl_json_gen_genewiz_rnaseq.sh
#!/bin/bash
ORDER=genewiz_rnaseq
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/jsons \
    -t rna-seq \
    --mem 48000 \
    --nthreads 24 \
    --separate-jsons \
    --skip-star-2pass \
    --read-length 150

Overwriting /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/cwl_json_gen_genewiz_rnaseq.sh


Execute file to create JSON files

In [5]:
%%script --out blocking_job_str bash -s "$blocking_job"
#  --depend afterok:$1 \
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -pnew,all -o /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/genewiz_rnaseq_cwl_json_gen.out \
 /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/cwl_json_gen_genewiz_rnaseq.sh

Extract blocking job id

In [16]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for pe-unstranded-with-sjdb samples

In [6]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/genewiz_rnaseq-pe-unstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=cwl_rna_seq
#SBATCH --output=/data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/genewiz_rnaseq-pe-unstranded-with-sjdb-%a.out
#SBATCH --mail-user=aeb84@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=48000
#SBATCH --cpus-per-task=24

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/STAR-STAR_2.4.1a/bin/Linux_x86_64/:$PATH"
export PATH="/data/reddylab/software/subread-1.4.6-p4-Linux-x86_64/bin/:$PATH"
export PATH="/data/reddylab/software/bamtools-2.2.3/bin/:$PATH"

export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/tmpdirs/tmp-genewiz_rnaseq-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/tmpdirs/tmp-genewiz_rnaseq-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/genewiz_rnaseq-pe-unstranded-with-sjdb  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/RNA-seq_pipeline/pipeline-pe-unstranded-with-sjdb.cwl \
    /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/jsons/rna_seq_download_metadata.genewiz_rnaseq-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}.json

Writing /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/genewiz_rnaseq-pe-unstranded-with-sjdb.sh


Execute SLURM array master file

In [9]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p new,all \
 --depend afterok:$1 \
 --array 2-3 \
 /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/genewiz_rnaseq-pe-unstranded-with-sjdb.sh

Extract blocking job id

In [ ]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [10]:
%%writefile /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/generate_qc_cell_genewiz_rnaseq-pe-unstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/logs/qc_gen.genewiz_rnaseq-pe-unstranded-with-sjdb.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/genewiz_rnaseq-pe-unstranded-with-sjdb

python /data/reddylab/software/cwl/bin/generate_stats_rnaseq_paired_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.PBC.txt@@') \
> qc.txt

Writing /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/generate_qc_cell_genewiz_rnaseq-pe-unstranded-with-sjdb.sh


Generate QCs for genewiz_rnaseq-pe-unstranded-with-sjdb

In [11]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p new,all \
 /data/reddylab/Alex/collab/20190701_Matt/processing/rna_seq/scripts/generate_qc_cell_genewiz_rnaseq-pe-unstranded-with-sjdb.sh

Extract blocking job id

In [ ]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

### Move raw reads to Flowcells

In [2]:
%%bash
mkdir -p /data/reddylab/Flowcells/20190701_Matt/rna_seq/genewiz/
mkdir -p /data/reddylab/Flowcells/20190701_Matt/rna_seq/novogene/
cp /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.genewiz_rnaseq.txt \
    /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/00_fastq/
mv /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/00_fastq \
    /data/reddylab/Flowcells/20190701_Matt/rna_seq/genewiz/
ln -s /data/reddylab/Flowcells/20190701_Matt/rna_seq/genewiz/00_fastq \
    /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/00_fastq
#
cp /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/metadata/rna_seq_download_metadata.rnaseq.novogene_pcks9.txt \
    /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/Rawdata/
mv /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/Rawdata \
    /data/reddylab/Flowcells/20190701_Matt/rna_seq/novogene/
ln -s /data/reddylab/Flowcells/20190701_Matt/rna_seq/novogene/Rawdata \
    /data/reddylab/Alex/collab/20190701_Matt/data/rna_seq/raw_reads/Rawdata